Задаём функция get_profiles(). Вызовем её, чтобы построить профили пользователей со стоимостью привлечения.

In [10]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np


In [11]:
def get_profiles(sessions, orders, events, ad_costs, event_names=[]):

    # находим параметры первых посещений
    profiles = (
        sessions.sort_values(by=['user_id', 'session_start'])
        .groupby('user_id')
        .agg(
            {
                'session_start': 'first',
                'channel': 'first',
                'device': 'first',
                'region': 'first',
            }
        )
        .rename(columns={'session_start': 'first_ts'})
        .reset_index()
    )

    # для когортного анализа определяем дату первого посещения
    # и первый день месяца, в который это посещение произошло
    profiles['dt'] = profiles['first_ts'].dt.date
    profiles['month'] = profiles['first_ts'].astype('datetime64[M]')

    # добавляем признак платящих пользователей
    profiles['payer'] = profiles['user_id'].isin(orders['user_id'].unique())

    # добавляем флаги для всех событий из event_names
    for event in event_names:
        if event in events['event_name'].unique():
            profiles[event] = profiles['user_id'].isin(
                events.query('event_name == @event')['user_id'].unique()
            )

    # считаем количество уникальных пользователей
    # с одинаковыми источником и датой привлечения
    new_users = (
        profiles.groupby(['dt', 'channel'])
        .agg({'user_id': 'nunique'})
        .rename(columns={'user_id': 'unique_users'})
        .reset_index()
    )

    # объединяем траты на рекламу и число привлечённых пользователей
    ad_costs = ad_costs.merge(new_users, on=['dt', 'channel'], how='left')

    # делим рекламные расходы на число привлечённых пользователей
    ad_costs['acquisition_cost'] = ad_costs['costs'] / ad_costs['unique_users']

    # добавляем стоимость привлечения в профили
    profiles = profiles.merge(
        ad_costs[['dt', 'channel', 'acquisition_cost']],
        on=['dt', 'channel'],
        how='left',
    )

    # стоимость привлечения органических пользователей равна нулю
    profiles['acquisition_cost'] = profiles['acquisition_cost'].fillna(0)

    return profiles

In [12]:
visits = pd.read_csv('C:\\Users\\37544\\Downloads\\ch02_problems03_visits.csv')  # журнал сессий
purchases = pd.read_csv('C:\\Users\\37544\\Downloads\\ch02_problems02_orders.csv')  # покупки
ad_costs = pd.read_csv('C:\\Users\\37544\\Downloads\\ch02_problems02_costs.csv')  # траты на рекламу
events = None  # других событий нет, None — чтобы не «сломать» get_profiles()

In [13]:
# преобразование данных о времени
visits['session_start'] = pd.to_datetime(visits['session_start'])
purchases['event_dt'] = pd.to_datetime(purchases['event_dt'])

In [14]:
# разбиваем траты на рекламу по дням
min_date = datetime(2020, 7, 11)
max_date = datetime(2020, 8, 10)
days_num = (max_date - min_date).days

In [15]:
daily_costs = []
for index, values in ad_costs.iterrows():
    channel_name = values['channel']
    channel_costs = values['costs']
    avg_channel_costs = channel_costs / days_num
    for i in range(days_num):
        current_day = min_date + timedelta(days=i)
        daily_costs += [[current_day, channel_name, avg_channel_costs]]

In [16]:
# создаём датафрейм с тратами на рекламу по дням
daily_costs = pd.DataFrame(daily_costs, columns=['dt', 'channel', 'costs'])
daily_costs['dt'] = daily_costs['dt'].dt.date

In [17]:
users =  get_profiles(visits, purchases, events, daily_costs)

In [18]:
users.head(5)

,user_id,first_ts,channel,device,region,dt,month,payer,acquisition_cost
0,242074,2020-08-05 06:04:32,Organic,Mac,United States,2020-08-05,2020-08-05 06:04:32,False,0.000000
1,4284686,2020-08-02 21:08:29,VaporStore,iPhone,United States,2020-08-02,2020-08-02 21:08:29,False,2.347259
2,10964006,2020-07-30 16:27:45,Organic,Android,Germany,2020-07-30,2020-07-30 16:27:45,False,0.000000
3,12358165,2020-07-26 02:06:13,Organic,Mac,United States,2020-07-26,2020-07-26 02:06:13,False,0.000000
4,12545358,2020-07-18 00:45:37,Organic,PC,United States,2020-07-18,2020-07-18 00:45:37,False,0.000000
